In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp fastai.learner

In [36]:
# export
# Python native modules
import os
from typing import *
# Third party libs
from fastcore.all import *
from torch.utils.data.dataloader_experimental import DataLoader2
import torchdata.datapipes as dp
from fastai.torch_basics import *
from fastai.torch_core import *
from torch.nn import *
from torch.optim import *
# Local modules
from fastrl.fastai.data.block import *
from fastrl.fastai.data.pipes.core import *

# Learner
> A revised fastai learner that uses DataPipe shimming

In [ ]:
# export
class XYSplit(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe,learn):
        self.source_datapipe = source_datapipe
        self.learn = learn
        
    def __iter__(self):
        for batch in self.source_datapipe:
            self.learn.xb = batch
            self.learn.preds = self.learn.model(*self.learn.xb)
            yield batch

In [43]:
# export
class ModelPredict(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe,learn):
        self.source_datapipe = source_datapipe
        self.learn = learn
        
    def __iter__(self):
        for batch in self.source_datapipe:
            self.learn.xb = batch
            self.learn.preds = self.learn.model(*self.learn.xb)
            yield batch

In [ ]:
# export
class LossCalc(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe,learn):
        self.source_datapipe = source_datapipe
        self.learn = learn
        
    def __iter__(self):
        for batch in self.source_datapipe:
            self.loss_grad = self.learn.loss_func(self.learn.pred, *self.yb)
            self.loss = self.loss_grad.clone()
            yield self.learn.preds

In [ ]:
# export
def default_train_loop(
    dls:List[DataLoader2],
    cbs:Optional[List[Callback]]=None,
):
    train_valid = L(dls).map(dp.iter.IterableWrapper).add_cbs(cbs)
    
    return train_vals

In [39]:
# export
def only_train_loop(
    dls:List[DataLoader2],
    cbs:Optional[List[Callback]]=None,
):
    train = dp.iter.IterableWrapper(dls).add_cbs(cbs)

    train = 
    
    return train

In [28]:
# export
class Learner(dp.iter.IterDataPipe):
    def __init__(self,model,dls,opt,loss_func,train_loop=None):
        store_attr()
        self.train_loop = ifnone(train_loop,default_train_loop)
        
    def fit(self,epochs):
        pass

In [22]:
# export
class DQN(Module):
    def __init__(self,state_sz:int,action_sz:int,hidden=512):
        self.layers=Sequential(
            Linear(state_sz,hidden),
            ReLU(),
            Linear(hidden,action_sz),
        )
    def forward(self,x): return self.layers(x)

In [23]:
block = DataBlock(
    blocks = GymTransformBlock,
    loader_loop=simple_iter_loader_loop
)
dls = block.dataloaders(['CartPole-v1']*4,n=20,n_workers=0)

In [26]:
model = DQN(4, 2)

In [40]:
learn = Learner(model, dls, opt=AdamW(model.parameters()), loss_func=MSELoss(),train_loop=only_train_loop)

In [19]:
# for o in dls[0]:
#     print(o)
#     break

In [12]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import *
    make_readme()
    notebook2script(silent=True)

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
